In [8]:
%load_ext autoreload
%autoreload 2

import itertools

from parsing.portfolio import Portfolio, SecurityGroup

import pandas as pd

from parsing.commodities import Brent
from parsing.currencies import Currency, __SELECTED__CURRENCIES__
from parsing.indexes import Index, __SELECTED_INDEXES__
from parsing.bonds import Bond, __SELECTED__BONDS__
from parsing.stocks import Stock, __SELECTED_STOCKS__
from parsing.zero_coupon import ZeroCoupon
import matplotlib.pyplot as plt
import seaborn as sns


plt.rcParams.update({'font.size': 14, 'figure.figsize': (12, 8)})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
brent = [Brent(period=period) for period in (1, 2, 3)]
currencies = [Currency(curr) for curr in __SELECTED__CURRENCIES__]
indexes = [Index(idx) for idx in __SELECTED_INDEXES__]
rates = ZeroCoupon()

In [10]:
stocks = [Stock(st) for st in __SELECTED_STOCKS__]
bonds = [Bond(b) for b in __SELECTED__BONDS__]

In [11]:
stock_group = SecurityGroup(name='stocks', securities=stocks, factors=brent + currencies + indexes, n_components=6)
currency_group = SecurityGroup(name='currencies', securities=currencies, factors=currencies, n_components=1)
bonds_group = SecurityGroup(name='bonds', securities=bonds, factors=brent + [rates], n_components=6)

In [12]:
portfolio = Portfolio(name='Factor 1-day portfolio', periods=1, factors=brent + currencies + indexes + [rates], securities=[stock_group, currency_group, bonds_group])

In [14]:
portfolio.fit()

  0%|          | 0/3 [00:00<?, ?it/s]

                            OLS Regression Results                            
Dep. Variable:                   ALRS   R-squared:                       0.257
Model:                            OLS   Adj. R-squared:                  0.256
Method:                 Least Squares   F-statistic:                     152.2
Date:                Mon, 25 Apr 2022   Prob (F-statistic):           1.11e-84
Time:                        01:26:42   Log-Likelihood:                 3608.1
No. Observations:                1322   AIC:                            -7208.
Df Residuals:                    1318   BIC:                            -7188.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0005      0.000      1.187      0.2

In [17]:
portfolio.returns()